# Presidential Election Data Preparation Notebook
This notebook takes the raw Presidential Elections spreadsheet in data/elections.csv and converts it into a linear list of records suitable
for analysis.  The spreadsheet is designed for readability -  {"name": "Years and candidates are in the columns, states are in the rows.  We need 
to convert this into a table of records (State, Year, Candidate, Party, Votes, Percentage of the Vote).  The result will be written to scratch/elections.sdml.
There is also a little data cleansing required.  The year row only has a year in the first entry of a year (e.g., 2020, '', '', '', 2016, '', '' instead of 2020, 2020, 2020, 2016, 2017, 2016).  The fields candidate and party are contained in a single cell of the form \<candidateName>   {"name": "\<party>.  Parties also have gone by different names over the years, and we add the column 'Canonical Party' to get the canonized name.  And integers are represented as strings "1,000,000" instead of 1000000







Read in the table from the CSV file.  After this, the data will be in the variable rows.

In [ ]:
import csv
with open('data/elections.csv', 'r') as f:
  election_reader = csv.reader(f)
  rows = [row for row in election_reader]


Define the schema of the table that will be built.  This will define the code below.  We want:
- Name: Name of the candidate
- Party: Candidate's party, as given at the time in the CSV file
- Canonical_Party: the canonical name of the party (handles party aliases)
- State
- Year: Year of the election
- Votes: votes received by the candidate in the state
- Total: Total vote in the state
- Percentage: Percentage of the vote received by the candidate

In [ ]:
schema = [
  {"name": "Name", "type": "string"},
  {"name": "Party", "type": "string"},
  {"name": "Canonical_Party", "type": "string"},
  {"name": "State", "type": "string"},
  {"name": "Year", "type": "number"},
  {"name": "Votes", "type": "number"},
  {"name": "Total", "type": "number"},
  {"name": "Percentage", "type": "number"}
]

Parties have gone by aliases over the years (e.g., "National Republican" for "Republican" or "Bull Moose" for "Progressive")

In [ ]:
PARTY_ALIASES = {
  'National Republican': "Republican",
  'National Union (Republican)': "Republican",
  'Whig': "Republican",
  'Liberal Republican/Democratic': "Democratic",
  '(Northern) Democratic': "Democratic",
  'Progressive "Bull Moose"':'Progressive'
}

Create a class to hold the votes and percentage of the vote received by a candidate in a state.  This will generate the (Votes, Total, Percentage) chunk of the record

In [ ]:
class CandidateStateRecord:
  # Votes, Total Votes, and Percentage of the Vote for a Candidate in a state
  def __init__(self, votes):
    self.votes = votes
    self.total = self.percentage = 0
  
  def compute_percentage(self, total):
    self.total = total
    self.percentage = 100 * self.votes/self.total
  
  def record(self):
    return [self.votes, self.total, self.percentage]



We'll create a class to accumulate the records.  This will have the candidate name, party, canonical party name, year, and a dictionary of votes by state.  Each instance will be initialized with the raw data in row 1, which is of the form `<name> - <party>`.  On initialization, it will also compute the canonical name of the party.

In [ ]:


class Candidate:
  def __init__(self, raw):
    raw = raw.strip()
    if raw == 'Total':
      self.name = 'Total'
      self.party = 'Total'
    else:
      parsed = raw.split(' - ')
      if len(parsed) == 2:
        self.name = parsed[0].strip()
        self.party = parsed[1].strip()
      else:
        self.name = ""
        self.party = "Other"
    self.canonical_party = PARTY_ALIASES[self.party] if self.party in PARTY_ALIASES else self.party
    self.year = 2020 # placeholder
    # create an empty dictionary for the votes
    self.state_record = {} # this will be a dictionary of CandidateStateRecord, indexed by state
    
  
  def records(self):
    # generate the records following the standard schema for this candidate
    return [[self.name, self.party, self.canonical_party, state, self.year] + vote_record.record() for (state, vote_record) in self.state_record.items()]
  

In [ ]:

candidate_row = rows[1][1:]
candidates = [Candidate(entry) for entry in candidate_row]

We now have a list of instances of Candidates.  The corresponding years are in row 0.  But some years are missing, so we need to fill those in

In [ ]:
year_row = rows[0][1:]
years = [year.strip() for year in year_row]
prev = years[0]
candidates[0].year = int(prev)
for i in range(1, len(years)):
  if years[i] == '':
    years[i] = prev
  else:
    prev = years[i]
  candidates[i].year = int(prev)

Now it's time to fill in the votes.  The states are in rows[2:], with the state name in column 1.  We won't bother adding in votes if the
vote is blank

In [ ]:
for row in rows[2:]:
  state = row[0]
  votes = row[1:]
  for i in range(len(votes)):
    try:
      candidates[i].state_record[state] = CandidateStateRecord(int(votes[i].strip().replace(",", "")))
    except ValueError:
      pass

Now, we want to group the candidates by state and year, and pull out the total votes for each state and year

In [ ]:
class StateYear:
  def __init__(self, state):
    # initialize with a candidate record
    self.candidates = []
    self.state = state
  
  def add_candidate(self, candidate):
    if candidate.name == 'Total':
      self.total = candidate.state_record[self.state].votes
    else:
      self.candidates.append(candidate)
  
  def compute_percentages(self):
    for candidate in self.candidates:
      candidate.state_record[self.state].compute_percentage(self.total)
      


Collect the distinct pairs state/year, pairs.  They will index a dictionary of candidate groups, which we will use to calculate percentages

In [ ]:

# Collect all the states and years
distinct_groups = []
for candidate in candidates:
  states = [(state, candidate.year) for state in candidate.state_record.keys() ]
  distinct_groups = distinct_groups + states
distinct_groups = list(set(distinct_groups))


Form the disctionary and add the candidates to the appropriate StateYear group in the dictionary

In [ ]:

groups = {}
for (state, year) in distinct_groups:
  groups[(state, year)] = StateYear(state)
for candidate in candidates:
  for state in candidate.state_record.keys():
    groups[(state, candidate.year)].add_candidate(candidate)

And now we can compute the percentages

In [ ]:
for state_year in groups.values(): state_year.compute_percentages()

We don't need the entries for the 'Total' candidate

In [ ]:
candidates = [candidate for candidate in candidates if candidate.name != 'Total']

The rows of the table are just the flattened rows of the candidates

In [ ]:
table_rows = [record for candidate in candidates for record in candidate.records()]

Take a look at the first few

In [ ]:
table_rows[:10]

Forming the table is easy

In [ ]:
import sdtp
table = sdtp.RowTable(schema, table_rows)

And now we can query the table.   Let's see how the Democrats did on the West Coast  from 1860 on

In [ ]:

year_filter = sdtp.GE('Year', 1860)
state_filter = sdtp.IN_LIST('State', ['Washington', 'Oregon', 'California'])
party_filter = sdtp.EQ('Canonical_Party', 'Democratic')
filter_spec = sdtp.ALL(year_filter, state_filter, party_filter)
table.get_filtered_rows(filter_spec, columns = ['State', 'Year', 'Percentage'])

We can get the results as another RowTable, in order to drill down

In [ ]:
result_table = table.get_filtered_rows(filter_spec, columns = ['State', 'Year', 'Percentage'], format ='sdml')
(result_table.schema, len(result_table.rows))

We can save the table

In [ ]:
import json
table_to_save = table.to_dictionary()
with open('scratch/presidential-elections.sdml', 'w') as f:
  json.dump(table_to_save, f, indent=2)

It's a good idea to check that the saved table is really the same as the original

In [ ]:
from sdtp import TableBuilder
with open('scratch/presidential-elections.sdml') as f:
  table_to_build = json.load(f)
table_from_disk = TableBuilder.build_table(table_to_build)


In [ ]:
assert table_from_disk.schema == table.schema
assert table_from_disk.rows == table.rows